In [3]:
RUNNING_LOCALLY = True

In [4]:
import os

# PLEASE SET YOUR OWN WORKING_DIRECTORY WHEN RUNNING LOCALLY
WORKING_DIRECTORY = "/home/yash/Desktop/Courses/CS2470/Final_Project/working_dir/"

if not RUNNING_LOCALLY:
    os.chdir("/home/yash/")
    print("Current Directory ->", os.getcwd())

    WORKING_DIRECTORY = os.getcwd()

    # Ensure that you are working in the right environment
    !echo $CONDA_PREFIX

In [5]:
import gym
import time
import numpy as np
from collections import deque
from gym import spaces
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

D:\program\Anaconda\envs\atari\lib\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
D:\program\Anaconda\envs\atari\lib\site-packages\keras_preprocessing\image\utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
D:\program\Anaconda\envs\atari\lib\site-packages\keras_preprocessing\image\utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
D:\program\Anaconda\envs\atari\lib\site-packages\keras_preprocessing\image\utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
D:\program\Anaconda\envs\atari\lib\site-packa

Num GPUs Available:  0


In [6]:
class ConcatObs(gym.Wrapper):
    def __init__(self, env, k):
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = \
            spaces.Box(low=0, high=255, shape=((k,) + shp), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        return np.array(self.frames)

In [7]:
# A bunch of wrappers to get us started, please use these
class ObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, GRAYSCALE=False, NORMALIZE=False):
        self.GRAYSCALE = GRAYSCALE
        self.NORMALIZE = NORMALIZE
        super().__init__(env)
    
    def observation(self, obs):
        # Normalise observation by 255

        
        if self.NORMALIZE:
            obs = obs / 255.0
        # Convert to grayscale -> This isn't quite working right now, but we can update the function quite easily later
        if self.GRAYSCALE:
#             obs = obs
            obs = tf.image.rgb_to_grayscale(obs)
                    
        image = obs[:,2:-9,8:,:]
        image = tf.image.resize(image,[84,84])
        image = tf.transpose(tf.reshape(image, image.shape[:-1]),perm = [1,2,0])
        return image

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, reward):
        # Clip reward between 0 to 1
        return np.clip(reward, 0, 1)
    
class ActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def action(self, action):
        return action

In [8]:
env = gym.make("ALE/Riverraid-v5")

In [12]:
# Using the wrappers for the environment
env = ObservationWrapper(RewardWrapper(ActionWrapper(ConcatObs(env, 4))), GRAYSCALE=True, NORMALIZE=True)
obs = env.reset()

In [116]:
#visualization
#image = tf.keras.preprocessing.image.array_to_img(tf.reshape(image[:,:,3],[84,84,1]))

**DQN**

In [14]:
hidden_size = 256
num_actions = 18
# q_net = tf.keras.Sequential()
# q_net.add(tf.keras.layers.Flatten())
# q_net.add(tf.keras.layers.Dense(hidden_size, activation='relu'))
# q_net.add(tf.keras.layers.Dense(num_actions, activation='softmax'))
#we can try to add convolutional layers

def create_q_model(input_shape, hidden_size, num_actions):
    # Network defined by the Deepmind paper
    inputs = layers.Input(shape=input_shape)
    # Convolutions on the frames on the screen
    
    layer1 = layers.Conv2D(16, 8, strides=4, activation="relu")(inputs)
    layer2 = layers.Conv2D(32, 4, strides=2, activation="relu")(layer1)
    layer3 = layers.Flatten()(layer2)
    layer4 = tf.keras.layers.Dense(hidden_size, activation='relu')(layer3) 
    action = tf.keras.layers.Dense(num_actions, activation='softmax')(layer4)

    return keras.Model(inputs=inputs, outputs=action)

q_net = create_q_model(input_shape=obs.shape, hidden_size=hidden_size, num_actions=num_actions)

In [15]:
trainer = tf.keras.optimizers.SGD(learning_rate = 0.01)

**One episode**

In [16]:
#hyper parameters
num_steps = 500
obs = env.reset()
rTot = 0
gamma = 0.99
e = 0.2

start_time = time.time()
for step in range(num_steps):

    with tf.GradientTape() as tape:
        # Compute the Q values and best action for the current state
        q_values = q_net(np.array([obs]))
        action = tf.math.argmax(q_values,axis=1).numpy()[0]
        
        # Epsilon adaptive
        if np.random.rand(1) < e:
            action = env.action_space.sample()

        # apply the action
        nst_obs, reward, done, _ = env.step(action)
        
        #SARSA: Same Policy for the new policy
        q_values_nst = q_net(np.array([nst_obs]))
        action_nst = tf.math.argmax(q_values_nst,axis=1).numpy()[0]
        if np.random.rand(1) < e:
            action_nst = env.action_space.sample()
        
        td_error = q_values.numpy()
        td_error[0][action] = reward + gamma*np.max(q_values_nst)
        loss = tf.reduce_sum(tf.square(td_error-q_values))
        
    grad = tape.gradient(loss,q_net.trainable_variables)
    trainer.apply_gradients(zip(grad,q_net.trainable_variables))
    rTot += reward
    
    obs = nst_obs
    
    if done:
        break
    
end_time = time.time()
inter = end_time-start_time

# Close the env
env.close()

In [17]:
print(rTot)
print(inter)

8.0
14.440385341644287


**Episodes with n_game**

In [157]:
num_steps = 500
obs = env.reset()
rTot = 0
gamma = 0.99
n_games = 10 #when running in the GCP, we can set a much higher value
E = 1000

rot_list = []
time_list = []
for i in range(0, n_games):
    print(f'Running game {i+1}/{n_games}...')
    
    e = E / (i + E)
    if e < 0.1:
        e = 0.1
        
    obs = env.reset()
    start_time = time.time()
    for step in range(num_steps):

        with tf.GradientTape() as tape:
            # Compute the Q values and best action for the current state
            q_values = q_net(np.array([obs]))
            action = tf.math.argmax(q_values,axis=1).numpy()[0]

            # Epsilon adaptive
            if np.random.rand(1) < e:
                action = env.action_space.sample()

            # apply the action
            nst_obs, reward, done, _ = env.step(action)


            q_values_nst = q_net(np.array([nst_obs]))
            action_nst = tf.math.argmax(q_values_nst,axis=1).numpy()[0]

            td_error = q_values.numpy()
            td_error[0][action] = reward + gamma*np.max(q_values_nst)
            loss = tf.reduce_sum(tf.square(td_error-q_values))

        grad = tape.gradient(loss,q_net.trainable_variables)
        trainer.apply_gradients(zip(grad,q_net.trainable_variables))
        rTot += reward

        obs = nst_obs

        if done:
            break

    end_time = time.time()
    inter = end_time-start_time
    rot_list.append(rTot)
    time_list.append(inter)

env.close()

Running game 1/100...
Running game 2/100...
Running game 3/100...
Running game 4/100...
Running game 5/100...
Running game 6/100...
Running game 7/100...
Running game 8/100...
Running game 9/100...
Running game 10/100...
Running game 11/100...
Running game 12/100...
Running game 13/100...
Running game 14/100...
Running game 15/100...
Running game 16/100...
Running game 17/100...
Running game 18/100...
Running game 19/100...
Running game 20/100...
Running game 21/100...
Running game 22/100...
Running game 23/100...
Running game 24/100...
Running game 25/100...
Running game 26/100...
Running game 27/100...
Running game 28/100...
Running game 29/100...
Running game 30/100...
Running game 31/100...
Running game 32/100...
Running game 33/100...
Running game 34/100...
Running game 35/100...
Running game 36/100...
Running game 37/100...
Running game 38/100...
Running game 39/100...
Running game 40/100...
Running game 41/100...
Running game 42/100...
Running game 43/100...
Running game 44/100.

In [158]:
print("average_reward:{}".format(sum(rot_list)/n_games))
print("average_time:{}".format(sum(time_list)/n_games))

average_reward:773.95
average_time:21.668797335624696


In [175]:
print("last_5_average_reward:{}".format(sum(rot_list[-5:])/5))
print("last_5_average_time:{}".format(sum(time_list[-5:])/5))

last_5_average_reward:1534.6
last_5_average_time:15.960728788375855


In [163]:
sum(time_list[-5:])/5

15.960728788375855

In [172]:
q_net.save("model/cDQN_100")

INFO:tensorflow:Assets written to: model/cDQN_100\assets


In [173]:
q_net_copy = keras.models.load_model("model/cDQN_100")

In [171]:
q_net_copy.get_weights()

[array([[[[ 5.56659326e-03,  6.29625320e-02,  5.11546060e-02, ...,
            5.17448969e-03,  3.33050154e-02,  4.81671840e-02],
          [-6.31109402e-02,  1.28968079e-02, -5.34964167e-02, ...,
           -1.84884947e-02, -7.00276643e-02,  1.47751002e-02],
          [ 1.87790040e-02,  6.30248524e-03,  5.18702604e-02, ...,
           -2.50804387e-02, -3.02724726e-02,  3.43974791e-02],
          [-4.31986749e-02,  1.88644156e-02,  2.03803694e-03, ...,
           -4.78029437e-02,  6.17231615e-03, -5.34662455e-02]],
 
         [[ 3.97651866e-02, -4.86325938e-03,  2.22705137e-02, ...,
            4.04938199e-02, -5.78751117e-02,  2.31522191e-02],
          [ 4.10063192e-02, -3.51383425e-02,  3.12007517e-02, ...,
           -3.11223753e-02, -2.12029032e-02, -4.65580672e-02],
          [-4.59631719e-02,  5.09581976e-02,  1.30186556e-03, ...,
            5.81130618e-03,  2.73961425e-02, -4.81345877e-02],
          [-1.16350185e-02, -4.35163677e-02, -6.43668026e-02, ...,
           -2.307144